<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL2.0/blob/main/07_mobile_web/query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#07. Mobile and web visitors
**With the following two tables, return the fraction of users who only visited mobile,only visited web, and visited both.**

In [1]:
!pip install polars
import pandas as pd
import numpy as  np
import polars as pl

In [2]:
data1 = {'user_id'   : [1,2,3,4,9,2,10],
         'page_url'  : ['A','B','C','A','B','C','B']
        }

data2 = {'user_id'    : [6,2,3,7,4,8,5],
         'page_url'   : ['A','B','C','A','B','C','B']
        }
pd_mobile=pd.DataFrame(data1)
pd_web=pd.DataFrame(data2)
pl_mobile=pl.LazyFrame(data1)
pl_web=pl.LazyFrame(data2)

In [3]:
print(f'----PANDAS----\nmobile table:\n{pd_mobile}')

----PANDAS----
mobile table:
   user_id page_url
0        1        A
1        2        B
2        3        C
3        4        A
4        9        B
5        2        C
6       10        B


In [4]:
print(f'web table:\n{pd_web}')

web table:
   user_id page_url
0        6        A
1        2        B
2        3        C
3        7        A
4        4        B
5        8        C
6        5        B


In [5]:
pd_df1=(pd.merge(pd_mobile.drop(columns='page_url')
                 ,pd_web.drop(columns='page_url')
                 ,on='user_id'
                 ,how='outer'
                 ,indicator=True
           )
          .drop_duplicates()
)
print(f'Identifying web or mobile users:\n{pd_df1}')

Identifying web or mobile users:
    user_id      _merge
0         1   left_only
1         2        both
3         3        both
4         4        both
5         5  right_only
6         6  right_only
7         7  right_only
8         8  right_only
9         9   left_only
10       10   left_only


In [6]:
pd_fractions=(pd.merge(pd_mobile.drop(columns='page_url')
                                .drop_duplicates()
                       ,pd_web.drop(columns='page_url')
                              .drop_duplicates()
                       ,on='user_id'
                       ,how='outer'
                       ,indicator=True
                 )['_merge']
                .value_counts(normalize=True)
                .rename(index={'right_only':'only_web',
                               'left_only':'only_mobile'})
)
print(f'fractions:')
pd_fractions

fractions:


,proportion
_merge,
only_web,0.4
only_mobile,0.3
both,0.3


In [7]:
quick1=pl_mobile.head(5).collect()
print(f'----POLARS----\nmobile table (sample):\n{quick1}')

----POLARS----
mobile table (sample):
shape: (5, 2)
┌─────────┬──────────┐
│ user_id ┆ page_url │
│ ---     ┆ ---      │
│ i64     ┆ str      │
╞═════════╪══════════╡
│ 1       ┆ A        │
│ 2       ┆ B        │
│ 3       ┆ C        │
│ 4       ┆ A        │
│ 9       ┆ B        │
└─────────┴──────────┘


In [8]:
quick2=pl_web.head(5).collect()
print(f'web table (sample):\n{quick2}')

web table (sample):
shape: (5, 2)
┌─────────┬──────────┐
│ user_id ┆ page_url │
│ ---     ┆ ---      │
│ i64     ┆ str      │
╞═════════╪══════════╡
│ 6       ┆ A        │
│ 2       ┆ B        │
│ 3       ┆ C        │
│ 7       ┆ A        │
│ 4       ┆ B        │
└─────────┴──────────┘


In [9]:
pl_lf1=(pl_mobile.head(5)
                 .select(pl.col('user_id'),
                         pl.col('user_id')
                           .alias('mobile_user')

                  )
                 .unique()
                 .join(pl_web.head(5)
                             .select(pl.col('user_id')
                                       .alias('web_user'),
                                     pl.col('user_id')
                              )
                             .unique(),
                       on='user_id',
                       how='full',
                       coalesce=True
                  )
).collect()
print(f'Identifying web or mobile users (sample):\n{pl_lf1}')

Identifying web or mobile users (sample):
shape: (7, 3)
┌─────────┬─────────────┬──────────┐
│ user_id ┆ mobile_user ┆ web_user │
│ ---     ┆ ---         ┆ ---      │
│ i64     ┆ i64         ┆ i64      │
╞═════════╪═════════════╪══════════╡
│ 6       ┆ null        ┆ 6        │
│ 3       ┆ 3           ┆ 3        │
│ 4       ┆ 4           ┆ 4        │
│ 7       ┆ null        ┆ 7        │
│ 2       ┆ 2           ┆ 2        │
│ 9       ┆ 9           ┆ null     │
│ 1       ┆ 1           ┆ null     │
└─────────┴─────────────┴──────────┘


In [10]:
pl_fractions=(pl_mobile.select(pl.col('user_id'),
                               pl.col('user_id')
                                 .alias('web_user')
                        )
                       .unique()
                       .join(pl_web.select(pl.col('user_id'),
                                           pl.col('user_id')
                                             .alias('mobile_user')

                                    )
                                   .unique(),
                             on='user_id',
                             how='full',
                             coalesce=True
                        )
                       .with_columns(mobile_user=pl.when(pl.col('mobile_user')
                                                           .is_null()
                                                    )
                                                   .then(1)
                                                   .otherwise(0),
                                     web_user=pl.when(pl.col('web_user')
                                                        .is_null()
                                                 )
                                                .then(1)
                                                .otherwise(0),
                                     both=pl.when(pl.col('mobile_user')==pl.col('web_user'))
                                            .then(1)
                                            .otherwise(0)
                       )
                      .select(pl.mean('web_user',
                                      'mobile_user',
                                      'both'
                                 )
                       )
).collect()
print(f'Fractions:')
pl_fractions

Fractions:


web_user,mobile_user,both
f64,f64,f64
0.4,0.3,0.3
